In [3]:
from abc import ABC, abstractmethod

In [5]:
class Runnable(ABC):
    @abstractmethod
    def invoke(input_data):
        """
        This method should be implemented by subclasses to define how the runnable processes input data.
        """
        pass

In [6]:
import random
class fakeLLM(Runnable):
    def __init__(self):
        print('LLM initialized')
    #mandatory method from Runnable class
    def invoke(self, prompt):
        
        response_list = [
            'Delhi is the capital of India.',
            'IPL is a popular cricket league.',
            'AI stand for Artificial Intelligence.',
        ]
        
        return{'response': random.choice(response_list)}
    
    def predict(self, prompt):

        response_list = [
            'Delhi is the capital of India.',
            'IPL is a popular cricket league.',
            'AI stand for Artificial Intelligence.',
        ]
        
        return{'response': random.choice(response_list)}

In [7]:
class fakePromptTemplate(Runnable):
    def __init__(self, template, input_variables):
        self.template = template
        self.input_variables = input_variables
    
    def invoke(self, input_dict):
        for var in self.input_variables:
            if var not in input_dict:
                raise ValueError(f"Missing input variable: {var}")
        # check for extra variables
        for key in input_dict:
            if key not in self.input_variables:
                raise ValueError(f"Unexpected input variable: {key}")
            
        return self.template.format(**input_dict)
    
    def format(self, input_dict):
        # check for missing variables
        for var in self.input_variables:
            if var not in input_dict:
                raise ValueError(f"Missing input variable: {var}")
        # check for extra variables
        for key in input_dict:
            if key not in self.input_variables:
                raise ValueError(f"Unexpected input variable: {key}")
            
        return self.template.format(**input_dict)

In [22]:
class fakeOutputParser(Runnable):

    def __init__(self):
        print('Output parser initialized')
        
    def invoke(self, output):
        if 'response' not in output:
            raise ValueError("Output must contain 'response' key")
        return output['response']
    
    def parse(self, output):
        if 'response' not in output:
            raise ValueError("Output must contain 'response' key")
        return output['response']

In [17]:
class RunnableConnector(Runnable):
    def __init__(self, Runnable_list):
        self.Runnable_list = Runnable_list

    def invoke(self, input_data):
        for runnable in self.Runnable_list:
            if isinstance(runnable, Runnable):
                input_data = runnable.invoke(input_data)
            else:
                raise TypeError(f"Expected a Runnable instance, got {type(runnable)}")
        return input_data

In [18]:
template = fakePromptTemplate(
    template = "Write a Poem about {topic}",
    input_variables = ['topic']
)

In [19]:
llm = fakeLLM()

LLM initialized


In [23]:
parser = fakeOutputParser()

Output parser initialized


In [24]:
chain = RunnableConnector([template, llm, parser])

In [25]:
chain.invoke({'topic': 'India'})

'IPL is a popular cricket league.'

In [27]:
template1 = fakePromptTemplate(
    template = "Write a joke about {topic}",
    input_variables = ['topic']
)

In [28]:
template2 = fakePromptTemplate(
    template = "Write a summary about following joke {response}",
    input_variables = ['response']
)

In [29]:
llm = fakeLLM()

LLM initialized


In [30]:
parser = fakeOutputParser()

Output parser initialized


In [35]:
chain1 = RunnableConnector([template1, llm])

In [36]:
chain2 = RunnableConnector([template2, llm, parser])

In [37]:
final_chain = RunnableConnector([chain1, chain2])

In [38]:
final_chain.invoke({'topic': 'Cricket'})

'Delhi is the capital of India.'